In [1]:
# Paso 3 con mapeo de cuantiles.
# Paso 2 sin mapeo de cuantiles.
# Obtiene la variable DNI (Direct Normal Irradiance)
# a partir de GHI usando el modelo DISC de NREL.

import numpy as np
import pandas as pd

import xarray as xr

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 1
c_lon = 1
Client( n_workers = 1, threads_per_worker = 1, memory_limit = "7GB" )

distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-23blj2hv', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 6.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49890,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 6.52 GiB
Comm: tcp://127.0.0.1:49895,Total threads: 1
Dashboard: http://127.0.0.1:49896/status,Memory: 6.52 GiB
Nanny: tcp://127.0.0.1:49893,


In [2]:
# Cargamos el archivo.
n = 8
path_1 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}_DNI.nc"
ds = xr.open_dataset( path_1, chunks = {
    "south_north": c_lat, "west_east": c_lon } )

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

# NREL DISC Model

# Huso horario.
# La información está en UTC.
TZ = 0

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day Angle"] = 6.283185 * ( ds["time"].dt.dayofyear - 1 ) / 365
# Extraterrestrial radiation * reciprocal of
# the square of the earth radius vector.
ds["ETR"] = ( 1367 * ( 1.00011 + 0.034221*np.cos(ds["Day Angle"])
    + 0.00128*np.sin(ds["Day Angle"]) + 0.000719*np.cos(2*ds["Day Angle"])
    + 0.000077*np.sin(2*ds["Day Angle"]) ) )
# Declinación
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day Angle"])
    + 0.070257*np.sin(ds["Day Angle"]) - 0.006758*np.cos(2*ds["Day Angle"])
    + 0.000907*np.sin(2*ds["Day Angle"]) - 0.002697*np.cos(3*ds["Day Angle"])
    + 0.00148*np.sin(3*ds["Day Angle"]) ) * 180/np.pi )
# Ecuación del tiempo.
ds["EQT"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day Angle"])
    - 0.032077*np.sin(ds["Day Angle"]) - 0.014615*np.cos(2*ds["Day Angle"])
    -0.040849*np.sin(2*ds["Day Angle"])) * 229.18 )
ds = ds.drop_vars( "Day Angle" )
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["time"].dt.hour - TZ + ds["EQT"]/60 )
# Ángulo horario.
ds["Hour Angle"] = ( 15 * ( ds["time"].dt.hour - 12
    - 0.5 + ds["EQT"]/60 + ((ds["lon"]-TZ*15)*4)/60 ) )
ds = ds.drop_vars( "EQT" )
# Posiciones del analema solar.
#ds["Sx"] = cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"])
#ds["Sy"] = ( cos(ds["lat"])*sin(ds["Declination"])
#    - sin(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
ds["Sz"] = ( sin(ds["lat"])*sin(ds["Declination"])
    - cos(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
ds = ds.drop_vars( "lon_subs" )
# Ángulo del cénit solar.
ds["Zenith Angle"] = acos(ds["Sz"])
ds = ds.drop_vars( "Sz" )
# Ángulo acimutal solar.
ds["Azimuth Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith Angle"])*sin(ds["lat"]) )
   / ( sin(ds["Zenith Angle"])*cos(ds["lat"]) ) )
ds["Azimuth Angle"] = ds["Azimuth Angle"].where(
    ds["Hour Angle"] < 0, 360 - ds["Azimuth Angle"] )
ds = ds.drop_vars( "Declination" )
# Masa de aire.
ds["Air Mass"] = ( 1/(cos(ds["Zenith Angle"])
    + 0.15/(93.885 - ds["Zenith Angle"])**1.253 ) * ds["Pressure"]/101325 )
ds["Air Mass"] = ds["Air Mass"].where( ds["Zenith Angle"] < 85.5, 0 )

ds.to_netcdf(path_2)

In [2]:
# Cargamos el archivo.
n = 8
path_1 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}_DNI.nc"
ds = xr.open_dataset( path_1, chunks = {
    "south_north": c_lat, "west_east": c_lon } )

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

# NREL DISC Model

# Huso horario.
# La información está en UTC.
TZ = 0

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day Angle"] = 6.283185 * ( ds["time"].dt.dayofyear - 1 ) / 365
# Extraterrestrial radiation * reciprocal of
# the square of the earth radius vector.
ds["ETR"] = ( 1367 * ( 1.00011 + 0.034221*np.cos(ds["Day Angle"])
    + 0.00128*np.sin(ds["Day Angle"]) + 0.000719*np.cos(2*ds["Day Angle"])
    + 0.000077*np.sin(2*ds["Day Angle"]) ) )
# Declinación
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day Angle"])
    + 0.070257*np.sin(ds["Day Angle"]) - 0.006758*np.cos(2*ds["Day Angle"])
    + 0.000907*np.sin(2*ds["Day Angle"]) - 0.002697*np.cos(3*ds["Day Angle"])
    + 0.00148*np.sin(3*ds["Day Angle"]) ) * 180/np.pi )
# Ecuación del tiempo.
ds["EQT"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day Angle"])
    - 0.032077*np.sin(ds["Day Angle"]) - 0.014615*np.cos(2*ds["Day Angle"])
    -0.040849*np.sin(2*ds["Day Angle"])) * 229.18 )
ds = ds.drop_vars( "Day Angle" )
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["time"].dt.hour - TZ + ds["EQT"]/60 )
# Ángulo horario.
ds["Hour Angle"] = ( 15 * ( ds["time"].dt.hour - 12
    - 0.5 + ds["EQT"]/60 + ((ds["lon"]-TZ*15)*4)/60 ) )
ds = ds.drop_vars( "EQT" )
# Posiciones del analema solar.
#ds["Sx"] = cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"])
#ds["Sy"] = ( cos(ds["lat"])*sin(ds["Declination"])
#    - sin(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
ds["Sz"] = ( sin(ds["lat"])*sin(ds["Declination"])
    - cos(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
ds = ds.drop_vars( "lon_subs" )
# Ángulo del cénit solar.
ds["Zenith Angle"] = acos(ds["Sz"])
ds = ds.drop_vars( "Sz" )
# Ángulo acimutal solar.
ds["Azimuth Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith Angle"])*sin(ds["lat"]) )
   / ( sin(ds["Zenith Angle"])*cos(ds["lat"]) ) )
ds["Azimuth Angle"] = ds["Azimuth Angle"].where(
    ds["Hour Angle"] < 0, 360 - ds["Azimuth Angle"] )
ds = ds.drop_vars( "Declination" )
# Masa de aire.
ds["Air Mass"] = ( 1/(cos(ds["Zenith Angle"])
    + 0.15/(93.885 - ds["Zenith Angle"])**1.253 ) * ds["Pressure"]/101325 )
ds["Air Mass"] = ds["Air Mass"].where( ds["Zenith Angle"] < 85.5, 0 )

ds.to_netcdf(path_2)

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function store_chunk at 0x123a2bf70>, (subgraph_callable-d9a87dde-0798-4669-8c2c-82fc547b4320, array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, F

RuntimeError: NetCDF: HDF error